# First Trial with PyVision

Credit: The original code for each section below is from the source the following source, however some modification is done to fit our purposes.

Sources: https://www.cs.colostate.edu/~cs510/yr2012/Progress/ProgressMedia/CSU_pyvision_Tutorial_CS510.pdf </br>
http://bolme.github.io/pyvision_docs/index.html

Import the package

In [1]:
import pyvision as pv
import cv2

Load an Image and show it.

In [9]:
im = pv.Image("sl4.jpg")
im.show()
cv2.destroyAllWindows() #This is just to close the image window

## Face Detection

In [18]:
import pyvision.face.CascadeDetector as cd
im = pv.Image("img/face.png")
face_detector = cd.CascadeDetector() #This class is a wrapper around the OpenCV cascade detectior.
#it returns faces collection since theree might be more than one face in the image.
faces = face_detector(im)
# or alternatively
#faces = face_detector.detect(im)

for face in faces:
    im.annotatePolygon(face.asPolygon(), width=4)

im.show(delay=0)
cv2.destroyAllWindows()

## Eye Detection

In [98]:
import pyvision.face.CascadeDetector as cd
import pyvision.face.FilterEyeLocator as ed
face_detect = cd.CascadeDetector()
eye_detect = ed.FilterEyeLocator()
im = pv.Image("img/me.jpg",bw_annotate=True)
faces = face_detect(im)
eyes = eye_detect(im,faces)
for face,eye1,eye2 in eyes:
    im.annotatePolygon(face.asPolygon(), width=4)
    im.annotatePoints([eye1,eye2])

im.show(delay=0)
cv2.destroyAllWindows()

## Face Cropping

In [16]:
import pyvision.face.CascadeDetector as cd
im = pv.Image("img/faceR.png")
face_detector = cd.CascadeDetector() #This class is a wrapper around the OpenCV cascade detectior.
faces = face_detector(im)
for face in faces:
    affine = pv.AffineFromRect(rect=face, new_Size=(255,255), interpolate=2) 
    cim = affine.transformImage(im)
    cim.show(delay=0)
    cv2.destroyAllWindows()

## Image Rotation
Image rotation is need because face detector will not work when the face is not in certain angle.
"Viola–Jones requires full view frontal upright faces". Wikipedia

In [29]:
im = pv.Image("img/faceR.png")
affine = pv.AffineRotate(theta=-3.14/2, new_size=(im.height,im.width), center= pv.Point(im.height/2,im.width/2),interpolate=2) 
cim = affine.transformImage(im)
cim.show(delay=0)
cv2.destroyAllWindows()

## Faces Alignment

In [66]:
import pyvision.face.CascadeDetector as cd
import pyvision.face.FilterEyeLocator as ed
face_detect = cd.CascadeDetector()
eye_detect = ed.FilterEyeLocator()

im2 = pv.Image("img/me.jpg")
im1 = pv.Image("img/face.png")

faces1 = face_detect(im1)
faces2 = face_detect(im2)
print eye_detect(im1,faces1)
face1,eye11,eye12 = eye_detect(im1,faces1)[0]
face2,eye21,eye22 = eye_detect(im2,faces2)[0]


affine = pv.AffineFromPoints(src1=eye11, src2=eye12, dst1=eye21, dst2=eye22, new_size=(im2.height,im2.width), interpolate=2)
cim1 = affine.transformImage(im1,im2)
cim1.show(delay=0)
im2.show(delay=0)
cv2.destroyAllWindows()

[[pv.Rect(83.076923,88.461538,186.923077,186.923077), pv.Point(138.569712,162.938702,0.000000), pv.Point(211.586538,162.938702,0.000000)]]


## Image Rescaling

In [35]:
im = pv.Image("img/face.png")
affine = pv.AffineScale(scale=0.5,  new_size=(256,256),interpolate=2) 
cim = affine.transformImage(im)
cim.show(delay=0)
cv2.destroyAllWindows()

## Image Perturbation

Slightly perturbs the image to generate more training images. images generated from this code should be used in the training with the same label of the original image.

In [97]:
im = pv.Image("img/face.png")

for i in xrange(10):
    affine = pv.AffinePerturb(Dscale =0.3, Drotate=0.56, Dtranslate=100, new_size=(256,256), mirror=False, flip=False, rng=None)
    cim = affine.transformImage(im)
    cv2.imwrite('img/face'+str(i)+'.png',cim.asOpenCV2())
    
# TODO we have to find out how to connect all the new image with the label of the original image


## Applying all preprocessing techniques

#### Can be found on pre.py file